In [1]:
import pickle
import nltk
import string
import numpy as np
import pandas as pd
import regex as re
import tensorflow as tf
from nltk.corpus import stopwords, words
from nltk.tokenize import word_tokenize,wordpunct_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from keras.models import load_model
from keras_preprocessing.sequence import pad_sequences
import time

pd.set_option('display.max_colwidth', 140)

In [2]:
import nltk.corpus
nltk.download('words')
words = words.words()
newWords = ['url','retweet','usermention','hashtag']
words.extend(newWords)
words = set(words)
stop_words = set(stopwords.words("english"))
wn = WordNetLemmatizer()

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Samar\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [3]:
ps =PorterStemmer()
start1 = time.time()
a = ps.stem("carries")
b = ps.stem("carried")
c = ps.stem("carrying")
end1 = time.time()
print("url :", a) 
print("retweet :", b) 
print("usermention :", c)
print("Time Taken: ", end1-start1)

url : carri
retweet : carri
usermention : carri
Time Taken:  0.0


In [4]:
start2 = time.time()
d = wn.lemmatize("carries")
e = wn.lemmatize("carried")
f = wn.lemmatize("carrying")
end2 = time.time()
print("url :", d) 
print("retweet :", e) 
print("usermention :", f) 
print("Time Taken: ", end2-start2)

url : carry
retweet : carried
usermention : carrying
Time Taken:  4.754691123962402


In [5]:
"hashtag" in words

True

In [6]:
print("url :", wn.lemmatize("url", pos='v')) 
print("retweet :", wn.lemmatize("retweet", pos='v')) 
print("usermention :", wn.lemmatize("usermention", pos='v')) 
print("hashtag :", wn.lemmatize("hashtag", pos='v')) 

url : url
retweet : retweet
usermention : usermention
hashtag : hashtag


In [7]:
ham_f1 = open("ham/ham1.txt", errors = 'ignore')
input_file1 = ham_f1.read()

In [8]:
ham_f2 = open("ham/ham2.txt", errors = 'ignore')
input_file2 = ham_f2.read()

In [9]:
ham_f3 = open("ham/ham3.txt", errors = 'ignore')
input_file3 = ham_f3.read()

In [10]:
ham_f4 = open("ham/ham4.txt", errors = 'ignore')
input_file4 = ham_f4.read()

In [11]:
ham_f5 = open("ham/ham5.txt", errors = 'ignore')
input_file5 = ham_f5.read()

In [12]:
ham_f6 = open("ham/ham6.txt", errors = 'ignore')
input_file6 = ham_f6.read()

In [13]:
ham_f7 = open("ham/ham7.txt", errors = 'ignore')
input_file7 = ham_f7.read()

In [14]:
spam_f1 = open("spam/spam1.txt", errors = 'ignore')
input_file8 = spam_f1.read()

In [15]:
spam_f2 = open("spam/spam2.txt", errors = 'ignore')
input_file9 = spam_f2.read()

In [16]:
spam_f3 = open("spam/spam3.txt", errors = 'ignore')
input_file10 = spam_f3.read()

In [17]:
spam_f4 = open("spam/spam4.txt", errors = 'ignore')
input_file11 = spam_f4.read()

In [18]:
spam_f5 = open("spam/spam5.txt", errors = 'ignore')
input_file12 = spam_f5.read()

In [19]:
def text_to_df(file):
    parsedData = file.replace('\t', '\n').split('\n')
    textList = parsedData[2::4]
    df = pd.DataFrame({'Text': textList})
    return df

In [20]:
tweets_ham1 = text_to_df(input_file1)
tweets_ham1.head()

,Text
0,â€¦ at house party in Daybreak. Not as weird as I'd feared. ;)
1,Taxiing â€” at SLC Salt Lake City International http://gowal.la/s/b7V
2,Almost home! â€” at PDX Portland International http://gowal.la/s/66X
3,Lunch! â€” at Ten01 http://gowal.la/s/Awa
4,Mm â€¦ books â€” at @Powells http://gowal.la/s/6fe


In [21]:
tweets_ham2 = text_to_df(input_file2)
tweets_ham2.head()

,Text
0,Woowooo Kentucky
1,Mom keeps talks about how complete shit New Moon was. Ahhh lovely.
2,#imthankfulfor my brother who I get to see today!!!!!!!!
3,Happy Thanksgiving!!! Macy's Parade! Watch for the Adair Co. Band !! That's my small ass town. Wooo go Kentucky.
4,@ad_ho didn't i say that


In [22]:
tweets_ham3 = text_to_df(input_file3)
tweets_ham3.head()

,Text
0,my house has fleas! I'm itchy all over
1,http://twitpic.com/b9r6p -
2,a bit bored
3,"thinking about reading HP 7 over the next week, forgot a lot of it"
4,HP 6 was really good


In [23]:
tweets_ham4 = text_to_df(input_file4)
tweets_ham4.head()

,Text
0,"@bevtales You're not supposed to legally cross Ortigas from Unimart to the bank side na, they constructed a footbridge in front of ibank!"
1,@bevtales Haha! You did't use the pedestrian flyover thingie by Caltex no?
2,@francepinzon @denisedahlin @Meryllchavez Whoaaa! Kelan 'to?
3,@katetorralba Guess which one... ? :p Who does it sound like?
4,@f_yao Came from HK. Internet withdrawal!


In [24]:
tweets_ham5 = text_to_df(input_file5)
tweets_ham5.head()

,Text
0,RT @dmbrown111: @Twithug @ELB55///TYVM A lister :):) @twithug back at ya//ppl like u make twitter worthwhile:)
1,"it's such a disgarce this BHO is going on 74 days(?) , presented w/several plans/options, and still NO decision &leaves country again!!#..."
2,@logoscoaching @scm15010 @Celinna @flexingtwon @Chimi3 @dmbrown111 @ri4dc @sonchief @TalulahStarr #FF
3,RT @leemayberry: #FollowFriday @ELB55///:)
4,"RT @oxco: RT @ELB55 LOL, PC anyone??? ""There's this thing called being so open-minded your brains drop out.""Richard Dawkins #tcot"


In [25]:
tweets_ham6 = text_to_df(input_file6)
tweets_ham6.head()

,Text
0,"2009-11-27 ä¸Šãƒ»ä¸­ç´šå…¬å‹™å“¡è©¦é¨“ éŽåŽ»å•ãƒ€ã‚¤ãƒ¬ã‚¯ãƒˆãƒŠãƒ“ æ€æƒ³,æ–‡å­¦ãƒ»èŠ¸è¡“[2011â€¦ è³‡æ ¼è©¦é¨“ç ”ç©¶ä¼š [å˜è¡Œæœ¬ï¼ˆã‚½ã..."
1,2009-11-27 é‹¼æ®»ã®ãƒ¬ã‚®ã‚ªã‚¹ç¬¬9å·» (é€šå¸¸ç‰ˆ)[DVD] å²¡æœ¬ä¿¡å½¦/é«˜åž£â€¦ http://bit.ly/4pBA0k
2,2009-11-27 ã‚¹ã‚¤ãƒ¼ãƒ„ãƒ¢ãƒãƒ¼ãƒ•ã®ã‚«ãƒ«ãƒˆãƒŠãƒ¼ã‚¸ãƒ¥--ãƒ™ãƒ«ã‚®ãƒ¼ã‹ã‚‰å±Šã„ãŸæ‰‹ä½œã‚Šã®â€¦ ä½ä¼¯ çœŸç´€ [å¤§åž‹æœ¬] http://bit.ly...
3,2009-11-27 æœ¬å½“ã«ã‚ã£ãŸã¾ã‚‹ç”Ÿã‚®ãƒ§ãƒ¼ãƒ†ãƒ³å† å©šè‘¬ç¥­SP (ã¾ã‚“ãŒã‚¿ã‚¤ãƒ ãƒžã‚¤ãƒ‘ãƒ«â€¦ å®‰æ–Ž ã‹ãªãˆ [ã‚³ãƒŸãƒƒã‚¯] http://bit....
4,2009-11-27 è„³ã¨å¿ƒã®æ´—ã„æ–¹ è‹«ç±³åœ°è‹±äºº [æ–°æ›¸] http://bit.ly/3qQmXe


In [26]:
tweets_ham7 = text_to_df(input_file7)
tweets_ham7.head()

,Text
0,@FeehAndrade quem te falo que eu chorei?! e nao to falando de hist. e sim de port.
1,@chillibeans http://twitpic.com/pvvse - Lindo *-*
2,"Triste demais pra twittar, so vim dar um oi ...oi <: )"
3,Nossa me enpolguei cantando ''No veneno'' ai a internet caiu... -.- -_-
4,@mikamasen OMG! eu sei! ta tuudo super esquisito !!


In [27]:
tweets_ham1.describe()

,Text
count,500000
unique,495245
top,@tarot3
freq,47


In [28]:
tweets_spam1 = text_to_df(input_file8)
tweets_spam1.head()

,Text
0,MELBOURNE ENQUIRY: Seeking a variety of acts for our end of year show. Payment is $120 per slot or $200 for 2.... http://bit.ly/4Ah3fF
1,THE BURLESQUE BOOTCAMP SYDNEY - Open Date tickets now available from http://bbootcampsyd.eventbrite.com/ for Jan /... http://bit.ly/rCenZ
2,THE BURLESQUE BOOTCAMP SYDNEY - Open Date tickets now available from http://bbootcampsyd.eventbrite.com/ for Jan /... http://bit.ly/1v5hvb
3,THE BURLESQUE BOOTCAMP SYDNEY - Open Date tickets now available from http://bbootcampsyd.eventbrite.com/ for Jan /... http://fb.me/3rzipF0
4,"Come to ""The Burlesque Bootcamp - Sydney"" Saturday, 23 January 2010 at 10:00 until <br />Sunday, 24 January 2010 at... http://bit.ly/38simD"


In [29]:
tweets_spam2 = text_to_df(input_file9)
tweets_spam2.head()

,Text
0,Am I not following you? Should I be? Reply to @JewTubeDotCom and I will follow.
1,@RBMORD Thank you! I'm following.
2,"@markjoyner Thanks for the follow. Just re-read (err...listened on Audible) ""Simpleology"". Happy Friday!"
3,Good Shabbos
4,rt @RabbiYonah Be yourself. Be Jewlicious. Feb 19-21 - tickets selling fast. http://jewliciousfestival.com/


In [30]:
tweets_spam3 = text_to_df(input_file10)
tweets_spam3.head()

,Text
0,Do You Have The Tools To Succeed Online? Visit http://www.homebizinstitute.com/launchblog/ One Place One Purpose
1,We're almost done... get ready for the one stop destination you've been waiting for... visit http://www.homebizinstitute.com/launchblog/
2,"New Bloganize Post: Yes, Most Facebook Users Are White http://bloganize.com/internet-traffic/yes-most-facebook-users-are-white/"
3,New Bloganize Post: Recropical Link Cloaking â€“ What you Need to Know http://bit.ly/7Ty81p
4,Are you sick of all the online marketing hype? A new way is coming... Learn & Earn at the http://www.homebizinstitute.com/launchblog/


In [31]:
tweets_spam4 = text_to_df(input_file11)
tweets_spam4.head()

,Text
0,France travel information and travel guide.http://bit.ly/4UuEz4
1,"Search hundreds of travel sites, compare results..http://bit.ly/4UuEz4"
2,100 travel websites for your information.http://bit.ly/4UuEz4
3,Cheap travel insurance package.http://bit.ly/4UuEz4
4,"Reviews of vacations, hotels, resorts, vacation and travel ..."


In [32]:
tweets_spam5 = text_to_df(input_file12)
tweets_spam5.head()

,Text
0,Additional Income Stream. www.worldwidetrend.net
1,80% of Business comes from Outside the U.S. in Network Marketing. www.WorldWideTrend.Net
2,Canada We Are Coming! www.worldwidetrend.net
3,"World Wide ""Brand"" ICON www.worldwidetrend.net"
4,"Marketing is Fun . . . ""Trump Network Wants You"": http://animoto.com/s/Zh1xVH8p8HrsDvVpaPwVyg"


In [33]:
tweets_spam1.describe()

,Text
count,499999
unique,455651
top,"Get thousand of followers for FREE! VIP are getting over 1,000 per day!! http://tinyurl.com/lxx4su"
freq,105


In [36]:
def clean_data(trial_text):
    for i in trial_text.index:
        trial_text.at[i,"Text"] = re.sub(r"http\S+", "url", trial_text.at[i,"Text"])
        trial_text.at[i,"Text"] = re.sub(r"@\S+", "usermention", trial_text.at[i,"Text"])
    trial_text.drop_duplicates(subset='Text', keep='first', inplace=True)
    for i in trial_text.index:
        trial_text.at[i,"Text"] = re.sub(r"#", "", trial_text.at[i,"Text"])
        trial_text.at[i,"Text"] = re.sub(r"\bRT\b", "retweet", trial_text.at[i,"Text"])
        trial_text.at[i,"Text"] = ' '.join(word for word in wordpunct_tokenize(trial_text.at[i,"Text"]) 
                                           if word.lower() not in stop_words)
        trial_text.at[i,"Text"] = ' '.join(word.strip(string.punctuation) for word in trial_text.at[i,"Text"].split())
        trial_text.at[i,"Text"] = wordpunct_tokenize(trial_text.at[i,"Text"].lower())
        trial_text.at[i,"Text"] = ' '.join([wn.lemmatize(word, pos='v') for word in trial_text.at[i,"Text"]])
        trial_text.at[i,"Text"] = ' '.join(word for word in word_tokenize(trial_text.at[i,"Text"]) if word in words)
        trial_text.at[i,"Text"] = " ".join(word for word in word_tokenize(trial_text.at[i,"Text"]) 
                                           if not (word.isalpha() and len(word)<3))
    
    #Additional loop for processing glove dataset
    for i in trial_text.index:
        trial_text.at[i,"Text"] = re.sub(r"\burl\b", "<url>", trial_text.at[i,"Text"])
        trial_text.at[i,"Text"] = re.sub(r"\busermention\b", "<user>", trial_text.at[i,"Text"]) 
        trial_text.at[i,"Text"] = re.sub(r"\bhashtag\b", "<hashtag>", trial_text.at[i,"Text"])
        trial_text.at[i,"Text"] = re.sub(r"\bretweet\b", "rt", trial_text.at[i,"Text"])
    trial_text['num_words']=trial_text["Text"].str.split().str.len()
    trial_text = trial_text.drop(trial_text[trial_text.num_words < 2].index)
    trial_text = trial_text.drop(['num_words'], axis=1)
    return trial_text

In [37]:
start2 = time.time()
clean_ham_df1 = clean_data(tweets_ham1)
clean_ham_df1.head()
end2 = time.time()
print("Time Taken: ", end2-start2)
clean_ham_df1.describe()

Time Taken:  489.28662943840027


,Text
count,357182
unique,310091
top,rt <user>
freq,2859


In [38]:
clean_ham_df2 = clean_data(tweets_ham2)
clean_ham_df2.head()

,Text
1,keep talk complete new moon lovely
2,brother get see today
3,happy thanksgiving parade watch band small ass town
4,<user> say
5,<user> say look like lipstick run


In [39]:
clean_ham_df3 = clean_data(tweets_ham3)
clean_ham_df3.head()

,Text
0,house itchy
2,bite bore
3,think read next week forget lot
4,really good
5,get blood test


In [40]:
clean_ham_df4 = clean_data(tweets_ham4)
clean_ham_df4.head()

,Text
0,<user> suppose legally cross bank side construct footbridge front
1,<user> use pedestrian
2,<user> <user> <user>
3,<user> guess one sound like
4,<user> come withdrawal


In [41]:
clean_ham_df5 = clean_data(tweets_ham5)
clean_ham_df5.head()

,Text
0,rt <user> <user> <user> lister <user> back like make twitter
1,days present several plan still decision leave country
2,<user> <user> <user> <user> <user> <user> <user> <user> <user>
3,rt <user> <user>
4,rt <user> rt <user> anyone thing call open mind brain drop


In [42]:
clean_ham_df6 = clean_data(tweets_ham6)
clean_ham_df6.head()

,Text
6,mac snow leopard <url>
7,final <url>
13,vol <url>
29,video game <url>
30,video game <url>


In [43]:
clean_ham_df7 = clean_data(tweets_ham7)
clean_ham_df7.head()

,Text
0,<user> hist port
1,<user> <url> lindo
2,vim dar
4,<user> super
5,<user> twitter


In [44]:
clean_ham_df1.to_csv('ham_glove/ham1.csv', sep='\t', index=False, header=False)
clean_ham_df2.to_csv('ham_glove/ham2.csv', sep='\t', index=False, header=False)
clean_ham_df3.to_csv('ham_glove/ham3.csv', sep='\t', index=False, header=False)
clean_ham_df4.to_csv('ham_glove/ham4.csv', sep='\t', index=False, header=False)
clean_ham_df5.to_csv('ham_glove/ham5.csv', sep='\t', index=False, header=False)
clean_ham_df6.to_csv('ham_glove/ham6.csv', sep='\t', index=False, header=False)
clean_ham_df7.to_csv('ham_glove/ham7.csv', sep='\t', index=False, header=False)

In [45]:
clean_ham_df1.describe()

,Text
count,357182
unique,310091
top,rt <user>
freq,2859


In [46]:
frames1 = [clean_ham_df2, clean_ham_df3, clean_ham_df4, clean_ham_df5, clean_ham_df6, clean_ham_df7]


clean_ham_df = clean_ham_df1.append(frames1, ignore_index=True)
clean_ham_df.describe()

,Text
count,2269340
unique,1788328
top,rt <user>
freq,16375


In [47]:
clean_ham_df.to_csv('ham_glove/ham.csv', sep='\t', index=False, header=False)

In [48]:
clean_spam_df1 = clean_data(tweets_spam1)
clean_spam_df1.head()

,Text
0,enquiry seek variety act end year show payment per slot <url>
1,burlesque open date ticket available <url> <url>
4,come burlesque <url>
5,century write perform black flamingo berlin <url>
6,burlesque come burlesque head <url>


In [49]:
clean_spam_df2 = clean_data(tweets_spam2)
clean_spam_df2.head()

,Text
0,follow reply <user> follow
1,<user> thank follow
2,<user> thank follow read err listen audible happy
4,<user> ticket sell fast <url>
5,<user> shalom thank add <url>


In [50]:
clean_spam_df3 = clean_data(tweets_spam3)
clean_spam_df3.head()

,Text
0,tool succeed visit <url> one place one purpose
1,almost get ready one stop destination wait visit <url>
2,new post yes white <url>
3,new post link cloak need know <url>
4,sick market new way come learn earn <url>


In [51]:
clean_spam_df4 = clean_data(tweets_spam4)
clean_spam_df4.head()

,Text
0,travel information travel guide <url>
1,search travel sit compare result <url>
2,travel information <url>
3,cheap travel insurance package <url>
4,review vacation resort vacation travel


In [52]:
clean_spam_df5 = clean_data(tweets_spam5)
clean_spam_df5.head()

,Text
0,additional income stream net
1,business come outside network market net
2,canada come net
3,world wide brand icon net
4,market fun trump network want <url>


In [53]:
clean_spam_df1.to_csv('spam_glove/spam1.csv', sep='\t', index=False, header=False)
clean_spam_df2.to_csv('spam_glove/spam2.csv', sep='\t', index=False, header=False)
clean_spam_df3.to_csv('spam_glove/spam3.csv', sep='\t', index=False, header=False)
clean_spam_df4.to_csv('spam_glove/spam4.csv', sep='\t', index=False, header=False)
clean_spam_df5.to_csv('spam_glove/spam5.csv', sep='\t', index=False, header=False)

In [54]:
frames2 = [clean_spam_df2, clean_spam_df3, clean_spam_df4, clean_spam_df5]


clean_spam_df = clean_spam_df1.append(frames2, ignore_index=True)
clean_spam_df.describe()
clean_spam_df.to_csv('spam_glove/spam.csv', sep='\t', index=False, header=False)

In [55]:
# Check length of longest tweet
x = clean_ham_df["Text"].str.split(" ")
max_len1 = 0
max_len2 = 0
max_len3 = 0
max_row1 = 0
count = 0
index = 0

for i in x:
    if len(i)>max_len1:
        max_len3 = max_len2
        max_len2 = max_len1
        max_len1 = len(i)
        max_row2 = max_row1
        max_row1 = i
        index = count
    count = count+1
        
print(max_len1)
print(max_len2)
print(max_len3)
print(index)
print(max_row1)
print(max_row2)

35
17
15
22142
['sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad']
['add', 'kill', 'achieve', 'world', 'domination', 'list', 'rt', '<user>', 'wonder', 'anyone', 'implement', 'shag', 'marry', 'murder', 'twitter', 'list', 'yet']


In [56]:
num = 0
for i in x:
    if len(i)==21:
        print(i)
        num+=1
print("number of length {} tweets: {}".format(len(i),num))

['rt', '<user>', 'want', 'bother', 'rt', '<user>', 'rt', '<user>', 'rt', '<user>', 'rt', '<user>', 'rt', '<user>', 'rt', '<user>', 'rt', '<user>', 'rt', '<user>', 'rt']
['ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', 'ring', '<url>']
['half', 'past', 'seven', 'half', 'past', 'seven', 'half', 'past', 'seven', 'half', 'past', 'seven', 'half', 'past', 'seven', 'half', 'past', 'seven', 'like', 'end', 'horror']
['<user>', 'everything', 'record', 'different', 'foo', 'feel', 'dont', 'sound', 'like', 'dope', 'man', 'dope', 'man', 'dont', 'sound', 'like', 'goon', 'get', 'rock', 'pop', 'country']
['team', 'cyrus', 'forever', 'team', 'cyrus', 'forever', 'team', 'cyrus', 'forever', 'team', 'cyrus', 'forever', 'team', 'cyrus', 'forever', 'team', 'cyrus', 'forever', 'team', 'cyrus', 'forever']
['team', 'cyrus', 'forever', 'team', 'cyrus', 'forever', 'team', 'cyrus', 'forever', 'team', 'cyrus', 'f

In [57]:
clean_ham_df['Label'] = 'Ham'
clean_ham_df.describe()

,Text,Label
count,2269340,2269340
unique,1788328,1
top,rt <user>,Ham
freq,16375,2269340


In [58]:
clean_spam_df['Label'] = 'Spam'
clean_spam_df.describe()

,Text,Label
count,1725268,1725268
unique,1480393,1
top,subscribe channel <url>,Spam
freq,2665,1725268


In [59]:
tweets_text = clean_ham_df.append(clean_spam_df, ignore_index=True)
tweets_text.describe()

,Text,Label
count,3994608,3994608
unique,3229606,2
top,rt <user>,Ham
freq,17325,2269340


In [60]:
#Summary of Tweet Text dataset
print("Out of {} rows, {} are spams, {} are non-spams".format(len(tweets_text),
                                                              len(tweets_text[tweets_text['Label']=='Spam']),
                                                              len(tweets_text[tweets_text['Label']=='Ham'])))

Out of 3994608 rows, 1725268 are spams, 2269340 are non-spams


In [61]:
tweets_text.isnull().sum()

Text     0
Label    0
dtype: int64

In [62]:
#export cleaned Tweet Text dataset
tweets_text.to_csv('final_dataset_glove.txt', sep='\t', index=False, header=False)